In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import torch
from bgtorch.utils.types import assert_numpy

In [30]:
from bgtorch.nn.training import KLTrainer

In [31]:
from bgtorch.nn import DenseNet
from bgtorch.nn.flow import (
    SequentialFlow, 
    CouplingFlow, 
    AffineFlow, 
    SplitFlow,
    MergeFlow,
    InverseFlow, 
    SwapFlow
)
from bgtorch.nn.flow.transformer import AffineTransformer

In [32]:
from bgtorch import BoltzmannGenerator

In [33]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

In [34]:
plt.rcParams.update({'font.size': 16})

In [35]:
from bgtorch.distribution.energy.base import Energy
from bgtorch.distribution.energy import DoubleWellEnergy
from bgtorch.distribution import NormalDistribution

In [36]:
from snf_code import InterpolatedEnergy, RNVP, boltzmann_generator_RNVP_MC
from snf_code.analysis import sample_energy, statistical_efficiency
from snf_code.imagetools import boltzmann_generator_NSF_MC

Basic functions
----

In [37]:
def sample_flow(flow, x0, inverse=False):
    blocks = flow._blocks
    samples = [x0]
    x = [x0]
    if inverse:
        blocks = blocks[::-1]
    for block in blocks:
        *x, ddlogp = block(*x, inverse=inverse)
        if not (isinstance(block, SplitFlow) or isinstance(block, SwapFlow) or isinstance(block, MergeFlow)):
            #print(block)
            x_np = [xi.detach().numpy() for xi in x]
            samples.append(np.hstack(x_np))
    if inverse:
        samples = samples[::-1]
    return samples

In [38]:
def plot_samples(samples, weights=None, range=None, ax=None):
    """ Plot sample histogram in 2D """
    samples = assert_numpy(samples)
    if ax is None:
        ax = plt.gca()
    ax.hist2d(
        samples[:, 0], 
        -samples[:, 1],
        weights=assert_numpy(weights) if weights is not None else weights,
        bins=100,
        norm=mpl.colors.LogNorm(),
        range=range
    )

In [39]:
def bias_uncertainty(target_energy, x, energies, w_x, w_energies):
    X = torch.Tensor(np.vstack([x, np.zeros((1, len(x)))]).T)
    E_target = target_energy.energy(X)[:, 0]
    E_target -= E_target.min()
    ######
    E_target = E_target.detach().numpy()

    # unweighted
    E_mean = np.mean(energies, axis=0)
    E_mean -= E_mean.min()

    # weighted
    Ew_mean = np.mean(w_energies, axis=0)
    Ew_mean -= Ew_mean.min()

    I = np.logical_and(hist_x > -2.25, hist_x < 2.25)
    # bias
    bias_unweighted = E_target - E_mean
    #bias_unweighted = bias_unweighted.detach().numpy()
    J = np.isfinite(bias_unweighted)
    bias_unweighted = np.abs(bias_unweighted[I*J].mean())
    bias_reweighted = E_target - Ew_mean
    #bias_reweighted = bias_reweighted.detach().numpy()
    J = np.isfinite(bias_reweighted)
    bias_reweighted = np.abs(bias_reweighted[I*J].mean())
    # uncertainty
    std_unweighted = np.array(energies)[:, I*J].std(axis=0).mean()
    std_reweighted = np.array(w_energies)[:, I*J].std(axis=0).mean()

    return bias_unweighted, std_unweighted, bias_reweighted, std_reweighted

In [40]:
def plot_energy(target_energy, x, energies, w_x, w_energies, ylabel=False, nstd=2.0, figsize=(4, 4)):
    fig = plt.figure(figsize=figsize)
    
    X = torch.Tensor(np.vstack([np.linspace(-3, 3, num=100), np.zeros((1, 100))]).T)
    E_target = target_energy.energy(X)
    E_target -= E_target.min()
    plt.plot(X[:, 0], E_target, linewidth=3, color='#444444')

    # unweighted
    E_mean = np.mean(energies, axis=0)
    E_mean -= E_mean.min()
    plt.errorbar(x, E_mean, nstd*np.std(energies, axis=0), color='red', linewidth=2)
    
    # weighted
    Ew_mean = np.mean(w_energies, axis=0)
    Ew_mean -= Ew_mean.min()
    plt.errorbar(w_x, Ew_mean, nstd*np.std(w_energies, axis=0), color='green', linewidth=2)
    
    plt.ylim(-1, 14)
    plt.xlabel('$x_1$')
    if ylabel:
        plt.ylabel('Energy (kT)')
    else:
        plt.yticks([])
    return fig

In [41]:
from bgtorch.nn.flow.base import Flow

class Adaptive_MetropolisMCFlow(Flow):
    def __init__(self, energy_model, nsteps=1, stepsize=0.01):
        """ Stochastic Flow layer that simulates Metropolis Monte Carlo

        """
        super().__init__()
        self.energy_model = energy_model
        self.nsteps = nsteps
        self.stepsize = torch.nn.Parameter(torch.ones(self.nsteps) * stepsize)
    
    def _forward(self, x, **kwargs):
        """ Run a stochastic trajectory forward 
        
        Parameters
        ----------
        x : PyTorch Tensor
            Batch of input configurations
        
        Returns
        -------
        x' : PyTorch Tensor
            Transformed configurations
        dW : PyTorch Tensor
            Nonequilibrium work done, always 0 for this process
            
        """
        E0 = self.energy_model.energy(x)
        E = E0

        for i in range(self.nsteps):
            # proposal step
            stepsize = torch.clamp(torch.abs(self.stepsize[i])+1e-2, 1e-2, 0.3) 
            dx = stepsize * torch.zeros_like(x).normal_()
            xprop = x + dx
            Eprop = self.energy_model.energy(xprop)
            
            # acceptance step
            acc = (torch.rand(x.shape[0], 1) < torch.exp(-(Eprop - E))).float()  # selection variable: 0 or 1.
            x = (1-acc) * x + acc * xprop
            E = (1-acc) * E + acc * Eprop

        # Work is energy difference
        dW = E - E0
        
        return x, dW

    def _inverse(self, x, **kwargs):
        """ Same as forward """
        return self._forward(x, **kwargs)


In [42]:
from snf_code import NSF

def boltzmann_generator_NSF_adaptiveMC(prior, target, n_transform, 
                               n_bins=20,
                               tail=1,
                               width_nhidden=[64, 64, 64], 
                               height_nhidden=[64, 64, 64],
                               slope_nhidden=[64, 64, 64],
                               stochastic=False, diffuse_at_0=False, nrelax=20, stepsize=0.1):
    # here we aggregate all layers of the flow
    layers = []

    # first flow
    if diffuse_at_0:
        layers.append(Adaptive_MetropolisMCFlow(prior, nsteps=nrelax, stepsize=stepsize))

    # split
    layers.append(SplitFlow(prior.dim//2))
        
    # RealNVP block
    for i in range(n_transform):
        # ic(x) -> v
        layers.append(NSF([prior.dim//2] + width_nhidden + [n_bins * prior.dim//2], 
                          [prior.dim//2] + height_nhidden + [n_bins * prior.dim//2], 
                          [prior.dim//2] + slope_nhidden + [(n_bins + 1) * prior.dim//2], 
                          width_activation=torch.nn.ReLU(),
                          height_activation=torch.nn.ReLU(),
                          slope_activation=torch.nn.ReLU(),
                          n_bins=n_bins,
                          tail=tail
                         ))
        layers.append(SwapFlow())
                            
        # v -> ic(x)
        layers.append(NSF([prior.dim//2] + width_nhidden + [n_bins * prior.dim//2], 
                          [prior.dim//2] + height_nhidden + [n_bins * prior.dim//2], 
                          [prior.dim//2] + slope_nhidden + [(n_bins + 1) * prior.dim//2], 
                          width_activation=torch.nn.ReLU(),
                          height_activation=torch.nn.ReLU(),
                          slope_activation=torch.nn.ReLU(),
                          n_bins=n_bins,
                          tail=tail
                         ))
        layers.append(SwapFlow())

        if stochastic and i < n_transform-1:
            layers.append(MergeFlow(prior.dim//2))
            
            lambda_ = i * 1.0/(n_transform-1)
            energy_model = InterpolatedEnergy(prior, target, lambda_)
            layers.append(Adaptive_MetropolisMCFlow(energy_model, nsteps=nrelax, stepsize=stepsize))
        
            layers.append(SplitFlow(prior.dim//2))

    # merge
    layers.append(MergeFlow(prior.dim//2))
    
    # final flow
    if stochastic:
        layers.append(Adaptive_MetropolisMCFlow(target, nsteps=nrelax, stepsize=stepsize))

    # now define the flow as a sequence of all operations stored in layers
    flexflow = SequentialFlow(layers)
    
    bg = BoltzmannGenerator(prior, flexflow, target)
    
    return bg

def boltzmann_generator_RNVP_adaptiveMC(prior, target, n_transform, shift_nhidden=[64, 64, 64], scale_nhidden=[64, 64, 64],
                                stochastic=False, diffuse_at_0=False, nrelax=20, stepsize=0.1):
    # here we aggregate all layers of the flow
    layers = []

    # first flow
    if diffuse_at_0:
        layers.append(Adaptive_MetropolisMCFlow(prior, nsteps=nrelax, stepsize=stepsize))

    # split
    layers.append(SplitFlow(prior.dim//2))
        
    # RealNVP block
    for i in range(n_transform):
        # ic(x) -> v
        layers.append(RNVP([prior.dim//2] + shift_nhidden + [prior.dim//2], 
                           [prior.dim//2] + scale_nhidden + [prior.dim//2], 
                           shift_activation=torch.nn.ReLU(), scale_activation=torch.nn.ReLU()))
        layers.append(SwapFlow())
                            
        # v -> ic(x)
        layers.append(RNVP([prior.dim//2] + shift_nhidden + [prior.dim//2], 
                           [prior.dim//2] + scale_nhidden + [prior.dim//2], 
                           shift_activation=torch.nn.ReLU(), scale_activation=torch.nn.ReLU()))
        layers.append(SwapFlow())

        if stochastic and i < n_transform-1:
            layers.append(MergeFlow(prior.dim//2))
            
            lambda_ = i * 1.0/(n_transform-1)
            energy_model = InterpolatedEnergy(prior, target, lambda_)
            layers.append(Adaptive_MetropolisMCFlow(energy_model, nsteps=nrelax, stepsize=stepsize))
        
            layers.append(SplitFlow(prior.dim//2))

    # merge
    layers.append(MergeFlow(prior.dim//2))
    
    # final flow
    if stochastic:
        layers.append(Adaptive_MetropolisMCFlow(target, nsteps=nrelax, stepsize=stepsize))

    # now define the flow as a sequence of all operations stored in layers
    flexflow = SequentialFlow(layers)
    
    bg = BoltzmannGenerator(prior, flexflow, target)
    
    return bg


System and Data
-----

In [43]:
prior = NormalDistribution(2)
target = DoubleWellEnergy(2, a=-0.5, b=-6)

In [44]:
from bgtorch.distribution.sampling.mcmc import GaussianMCMCSampler

In [45]:
sampler_left = GaussianMCMCSampler(target, noise_std=0.2, init_state=torch.Tensor([[-2, 0]]), n_stride=10)
x_left = sampler_left.sample(10000)

sampler_right = GaussianMCMCSampler(target, noise_std=0.2, init_state=torch.Tensor([[2, 0]]), n_stride=10)
x_right = sampler_right.sample(10000)

x_both = torch.cat([x_left, x_right], dim=0)

In [47]:
# exact sample
brute_sampler = GaussianMCMCSampler(target, init_state=torch.tensor([[-2., 0]]), noise_std=1.5, n_stride=1)
x_brute = brute_sampler.sample(100000)

Bias versus Variance
------

In [49]:
#data = x_both
data = x_brute

bg = boltzmann_generator_RNVP_adaptiveMC(prior, target, n_transform=3, shift_nhidden=[64, 64], scale_nhidden=[64, 64],
                                 stochastic=False)
trainer = KLTrainer(bg, train_likelihood=True, train_energy=True,
                    optim=torch.optim.Adam(bg.parameters(), lr=0.005))

trainer.train(300, data=data, batchsize=128, w_likelihood=1, w_energy=0, n_print=0)
trainer.train(300, data=data, batchsize=128, w_likelihood=0.5, w_energy=0.5, n_print=0)

In [50]:
hist_x, hists_y, whist_x, whists_y = sample_energy(bg, 100000, 20, nbins=30)

Train and analyze several times (statistics)
---

In [51]:
def train_and_analyze(bg, data):
    trainer = KLTrainer(bg, train_likelihood=True, train_energy=True,
                        optim=torch.optim.Adam(bg.parameters(), lr=0.005))

    trainer.train(300, data=data, batchsize=128, w_likelihood=1, w_energy=0, n_print=0)
    trainer.train(300, data=data, batchsize=128, w_likelihood=0.5, w_energy=0.5, n_print=0)
    #trainer_unbiased_MC.train(200, data=x_brute, batchsize=128, w_likelihood=0, w_energy=1, n_print=0)

    se, ses = statistical_efficiency(bg, n_samples=50000, n_resample=1000)

    hist_x, hists_y, whist_x, whists_y = sample_energy(bg, 100000, 20, nbins=30)
    bias, std, bias_w, std_w = bias_uncertainty(target, hist_x, hists_y, whist_x, whists_y)
    
    return [bias, std, bias_w, std_w]

In [52]:

# RNVP + MC unbiased
stats_RNVP_MC = []
for i in range(10):
    bg_unbiased_MC = boltzmann_generator_RNVP_adaptiveMC(prior, target, n_transform=3, shift_nhidden=[64, 64], scale_nhidden=[64, 64],
                                                 nrelax=20, stepsize=0.05, stochastic=True, diffuse_at_0=True)
    stat = train_and_analyze(bg_unbiased_MC, x_brute[::10])
    stats_RNVP_MC.append(stat)
    print(i, '\t', stat[0], '\t', stat[1], '\t', stat[2], '\t', stat[3])    
stats_RNVP_MC = np.array(stats_RNVP_MC)



0 	 0.7058265436867651 	 0.24285042561351253 	 0.03642463714201749 	 0.4001397529677428
1 	 1.20210292286347 	 0.17967980753823798 	 0.010697787218761912 	 0.3536563802698501
2 	 0.5983738072474113 	 0.2530745178524027 	 0.17192235488509597 	 0.4774365786097161
3 	 0.816241714034399 	 0.2512792582820743 	 0.012649543115463624 	 0.44707691434977975
4 	 0.9058474889640175 	 0.21817359538527378 	 0.06290970947061174 	 0.3853170302215556
5 	 1.013574467913352 	 0.18884381063325528 	 0.020318324777225893 	 0.35757088467448206
6 	 1.0969533669523523 	 0.24097178821179557 	 0.09411258169183633 	 0.4393284623887627
7 	 1.0423406237926374 	 0.2230604924877877 	 0.057316522940934866 	 0.45683455881278867
8 	 0.8411043582095621 	 0.23241985055627434 	 0.059627340863061505 	 0.41590843358074575
9 	 1.146682073189269 	 0.2142818060290248 	 0.0019930040226666854 	 0.40466536001841097


In [53]:

stats = stats_RNVP_MC
print('    Bias       Unc        Bias rew   Unc rew')
print('  ', np.mean(stats, axis=0))
print('+-', np.std(stats, axis=0))
print()
ee_unweighted = np.sqrt(stats[:, 0]**2+stats[:, 1]**2)
ee_reweighted = np.sqrt(stats[:, 2]**2+stats[:, 3]**2)
print('sqrt(bias**2+std**2)\t', np.mean(ee_unweighted), '+-', np.std(ee_unweighted))
print('reweighted          \t', np.mean(ee_reweighted), '+-', np.std(ee_reweighted))



    Bias       Unc        Bias rew   Unc rew
   [0.93690474 0.22446354 0.05279718 0.41379344]
+- [0.18698912 0.02372998 0.04832732 0.03940226]

sqrt(bias**2+std**2)	 0.9656603492113291 +- 0.17664228314941716
reweighted          	 0.4193425302211292 +- 0.04530408383793004
